In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler
import numpy as np
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

#turn off the warnings
import warnings
warnings.filterwarnings('ignore')

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df    

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Drop the non-beneficial ID columns.
#application_df = application_df.drop(columns=['USE_CASE', 'SPECIAL_CONSIDERATIONS', 'STATUS'])

In [4]:
#Explore Income Amt
application_df['INCOME_AMT'].unique()

array(['0', '1-9999', '10000-24999', '100000-499999', '10M-50M',
       '25000-99999', '50M+', '1M-5M', '5M-10M'], dtype=object)

In [5]:
#Replace the Income Ranges with Numbers 
application_df['INCOME_AMT'].replace(['0', '1-9999', '10000-24999', '100000-499999', '10M-50M',
       '25000-99999', '50M+', '1M-5M', '5M-10M'],[1,5000,13000,250000,25000000,50000,50000000,2500000,7500000], inplace=True)
application_df['INCOME_AMT'].unique()

array([       1,     5000,    13000,   250000, 25000000,    50000,
       50000000,  2500000,  7500000], dtype=int64)

In [6]:
#create new features
application_df['income_ratio']=(application_df['INCOME_AMT']/application_df['ASK_AMT'])
application_df = application_df.drop(columns=['ASK_AMT', 'INCOME_AMT'])

In [7]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
SPECIAL_CONSIDERATIONS       2
IS_SUCCESSFUL                2
income_ratio              8837
dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df.APPLICATION_TYPE.value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Visualize the value counts of APPLICATION_TYPE
#app_counts.plot.density()

In [10]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_counts[app_counts < 70].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
clas_counts = application_df.CLASSIFICATION.value_counts()
clas_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# Visualize the value counts of CLASSIFICATION
#clas_counts.plot.density()

In [13]:
# Determine which values to replace if counts are less than ..?
replace_class = list(clas_counts[clas_counts <100].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [14]:
# Merge Features                                        
#application_df['merged_field'] = application_df['ORGANIZATION']  + "_" +  application_df['AFFILIATION']
#application_df = application_df.drop(columns=['ORGANIZATION','AFFILIATION'])
#application_df

In [15]:
# Merge Features 2                                       
#application_df['merged_field2'] = str(application_df['AFFILIATION'])  + "_" +  application_df['ORGANIZATION']
#application_df = application_df.drop(columns=['AFFILIATION','ORGANIZATION'])
#application_df

In [16]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'SPECIAL_CONSIDERATIONS']

In [17]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [18]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,IS_SUCCESSFUL,income_ratio,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,0.000200,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,1,0.046045,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,0,0.000200,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,1.942618,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1,1,1.753279,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [19]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [20]:
# Create a StandardScaler instances
scaler = MinMaxScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 40
hidden_nodes_layer4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# 3rd hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# 3rd hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               8400      
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 40)                4040      
                                                                 
 dense_3 (Dense)             (None, 10)                410       
                                                                 
 dense_4 (Dense)             (None, 1)                 11        
                                                                 
Total params: 32,961
Trainable params: 32,961
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_Opt/",exist_ok=True)
checkpoint_path = "checkpoints_Opt/AlphabetSC.{epoch:02d}.hdf5"

In [23]:
train_size = len(X_train_scaled)
train_size 
#y_train.shape

25724

In [24]:
batch_size = 64
steps_per_epoch = train_size / batch_size
save_period = 5
save_freq = int(save_period * steps_per_epoch)

In [25]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq = save_freq)

In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, batch_size = batch_size, 
                   callbacks=[cp_callback])

Epoch 1/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5933 - accuracy: 0.7060
Epoch 2/100
402/402 [==============================] - 1s 3ms/step - loss: 0.5686 - accuracy: 0.7250
Epoch 3/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5639 - accuracy: 0.7266
Epoch 4/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5591 - accuracy: 0.7284
Epoch 5/100
391/402 [============================>.] - ETA: 0s - loss: 0.5566 - accuracy: 0.7306
Epoch 5: saving model to checkpoints_Opt\AlphabetSC.05.hdf5
402/402 [==============================] - 1s 2ms/step - loss: 0.5569 - accuracy: 0.7299
Epoch 6/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7282
Epoch 7/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5540 - accuracy: 0.7294
Epoch 8/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5536 - accuracy: 0.7287
Epoch 9/100
402/402 [==========================

402/402 [==============================] - 1s 2ms/step - loss: 0.5361 - accuracy: 0.7371
Epoch 65/100
373/402 [==========================>...] - ETA: 0s - loss: 0.5343 - accuracy: 0.7389
Epoch 65: saving model to checkpoints_Opt\AlphabetSC.65.hdf5
402/402 [==============================] - 1s 2ms/step - loss: 0.5359 - accuracy: 0.7371
Epoch 66/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5360 - accuracy: 0.7376
Epoch 67/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5353 - accuracy: 0.7375
Epoch 68/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5352 - accuracy: 0.7381
Epoch 69/100
402/402 [==============================] - 1s 2ms/step - loss: 0.5351 - accuracy: 0.7375
Epoch 70/100
378/402 [===========================>..] - ETA: 0s - loss: 0.5346 - accuracy: 0.7379
Epoch 70: saving model to checkpoints_Opt\AlphabetSC.70.hdf5
402/402 [==============================] - 1s 3ms/step - loss: 0.5353 - accuracy: 0.7376
Epoch 7

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5643 - accuracy: 0.7229 - 466ms/epoch - 2ms/step
Loss: 0.5642865300178528, Accuracy: 0.7229154706001282


In [29]:
# Export the model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")